In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

from PIL import Image
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import time
import random
from math import ceil
from tqdm import tqdm
import os

import torch.utils.data as data
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F


BATCH_SIZE = 10
DEVICE = torch.device('cpu')
NUM_EPOCHS = 10


In [10]:
train_data_ori = open("./E3_data/Short_train_labels.txt","r")

train_lines = train_data_ori.readlines()

train_data_new = torch.empty([0, 3, 64, 64])
train_label = []

i = 0
for line in train_lines:
    if i%20 == 0:
        print("load train dataset\t"+str(i))
    i += 1

    line_inf = line.split('\t')
    imgpath='./E3_data/train_img/'+line_inf[0]
    img = plt.imread(imgpath).astype(np.float32)
    img = torch.from_numpy(img).reshape([1, 3, 64, 64])
    train_data_new = torch.cat((train_data_new, img), 0)
 
    label = int(line_inf[1].replace("\n", ""))
    train_label.append(label)

train_label = np.array(train_label)
train_label = torch.from_numpy(train_label)


load train dataset	0
load train dataset	20
load train dataset	40
load train dataset	60
load train dataset	80
torch.Size([100, 3, 64, 64])


In [11]:
val_data_ori = open("./E3_data/Short_val_labels.txt","r")
val_lines = val_data_ori.readlines()

val_data_new = torch.empty([0, 3, 64, 64])
val_label = []

i = 0
for line in val_lines:
    if i%20 == 0:
        print("load val dataset\t"+str(i))
    i += 1

    line_inf = line.split('\t')
    imgpath='./E3_data/val_img/'+line_inf[0]
    img = plt.imread(imgpath).astype(np.float32)
    img = torch.from_numpy(img).reshape([1, 3, 64, 64])
    val_data_new = torch.cat((val_data_new, img), 0)
 
    label = int(line_inf[1].replace("\n", ""))
    val_label.append(label)
val_label = np.array(val_label)
val_label = torch.from_numpy(val_label)


load val dataset	0
load val dataset	20
load val dataset	40
load val dataset	60
load val dataset	80


In [12]:
test_data_ori = open("./E3_data/Short_Test_labels.txt","r")
test_lines = test_data_ori.readlines()

test_data_new = torch.empty([0, 3, 64, 64])
test_label = []

i = 0
for line in test_lines:
    if i%20 == 0:
        print("load test dataset\t"+str(i))
    i += 1

    line_inf = line.split('\t')
    imgpath='./E3_data/test_img/'+line_inf[0]
    img = plt.imread(imgpath).astype(np.float32)
    img = torch.from_numpy(img).reshape([1, 3, 64, 64])
    test_data_new = torch.cat((test_data_new, img), 0)
 
    label = int(line_inf[1].replace("\n", ""))
    test_label.append(label)
test_label = np.array(test_label)
test_label = torch.from_numpy(test_label)


load test dataset	0
load test dataset	20
load test dataset	40
load test dataset	60
load test dataset	80


In [13]:
train_dataset = data.TensorDataset(train_data_new, train_label)
val_dataset = data.TensorDataset(val_data_new, val_label)
test_dataset = data.TensorDataset(test_data_new, test_label)

train_loader = data.DataLoader(
    dataset=train_dataset,      # torch TensorDataset format
    batch_size=BATCH_SIZE,      # mini batch size
    shuffle=True,               # random shuffle for training
    num_workers=4,              # subprocesses for loading data
)

val_loader = data.DataLoader(
    dataset=val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
)

test_loader = data.DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
)

In [12]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(3,10,kernel_size=(5,5))
        self.conv2 = nn.Conv2d(10,20,kernel_size=(3,3))
        self.linear1 = nn.Linear(20*108*108,500)
        self.linear2 = nn.Linear(500,10)
        
    def forward(self, x):
        size_in = x.size(0)
        mid = self.conv1(x)
        mid = F.relu(mid)
        mid = F.max_pool2d(mid,2,2)
        mid = self.conv2(mid)
        mid = F.relu(mid)
        mid = mid.view(size_in,-1)
        mid = self.linear1(mid)
        mid = F.relu(mid)
        mid = self.linear2(mid)
        out = F.log_softmax(mid,dim = 1)
        return out 

model = SimpleNet()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [13]:
# train the model
for epoch in range(NUM_EPOCHS):
    print("epoch:", +epoch)
    model.train()
    for images, labels in tqdm(train_loader):
        images,labels = images.to(DEVICE),labels.to(DEVICE)
        optimizer.zero_grad()
        output = model(images)
        lossvalue = criterion(output,labels)
        lossvalue.backward()
        optimizer.step()
    # evaluate the model
    model.eval()
    test_loss = 0
    correct_num_test = 0
    train_loss = 0
    correct_num_train = 0
    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            output = model(images)
            train_loss += torch.nn.functional.nll_loss(output, labels, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct_num_train += pred.eq(labels.view_as(pred)).sum().item()
    train_loss /= len(test_loader.dataset)
    correct_rate = 100. * correct_num_train / len(train_loader.dataset)
    print('Train Accuracy: {0}/{1}({2:.4}%)'.format(correct_num_train,len(train_loader.dataset),correct_rate))

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            output = model(images)
            test_loss += torch.nn.functional.nll_loss(output, labels, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct_num_test += pred.eq(labels.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    correct_rate = 100. * correct_num_test / len(test_loader.dataset)
    print('Test  Accuracy: {0}/{1}({2:.4}%)'.format(correct_num_test,len(test_loader.dataset),correct_rate))   


100%|██████████| 10/10 [00:18<00:00,  1.81s/it]
Train Accuracy: 87/100(87.0%)
  0%|          | 0/10 [00:00<?, ?it/s]Test  Accuracy: 77/100(77.0%)
epoch: 1
100%|██████████| 10/10 [00:20<00:00,  2.01s/it]
Train Accuracy: 97/100(97.0%)
  0%|          | 0/10 [00:00<?, ?it/s]Test  Accuracy: 90/100(90.0%)
epoch: 2
100%|██████████| 10/10 [00:20<00:00,  2.02s/it]
Train Accuracy: 99/100(99.0%)
  0%|          | 0/10 [00:00<?, ?it/s]Test  Accuracy: 90/100(90.0%)
epoch: 3
100%|██████████| 10/10 [00:20<00:00,  2.08s/it]
Train Accuracy: 100/100(100.0%)
  0%|          | 0/10 [00:00<?, ?it/s]Test  Accuracy: 92/100(92.0%)
epoch: 4
100%|██████████| 10/10 [00:22<00:00,  2.20s/it]
Train Accuracy: 100/100(100.0%)
  0%|          | 0/10 [00:00<?, ?it/s]Test  Accuracy: 92/100(92.0%)
epoch: 5
100%|██████████| 10/10 [00:21<00:00,  2.11s/it]
Train Accuracy: 100/100(100.0%)
  0%|          | 0/10 [00:00<?, ?it/s]Test  Accuracy: 92/100(92.0%)
epoch: 6
100%|██████████| 10/10 [00:22<00:00,  2.20s/it]
Train Accuracy: 